### torchtext 라이브러리로 텍스트 분류
<hr>

- 1단계 : 데이터 전처리 : 숫자형식으로 변환하는 것 까지
- 2단계 : 모델 구현

- 1-1 데이터 준비 => 내장 데이터셋 활용
    * AG_NEWS 데이터셋 반복자 : 레이블 (label) + 문장의 튜플(tuple) 형태

In [2]:
# %env

In [3]:
%set_env NVIDIA_VISIBLE_DEVICES=MIG-da4f3553-5a31-5138-937b-a4a90f2ec139

env: NVIDIA_VISIBLE_DEVICES=MIG-da4f3553-5a31-5138-937b-a4a90f2ec139


In [4]:
import sys,os,os.path

os.environ['NVIDIA_VISIBLE_DEVICES']='MIG-da4f3553-5a31-5138-937b-a4a90f2ec139'

In [5]:
import torch



In [6]:
import torch
from torchtext.datasets import AG_NEWS

# ==> DataPipe 타입 >> iterator 타입 형변환
train_iter = iter(AG_NEWS(split='train'))



In [7]:
## 데이터 확인 => (label, text), label 1-4
next(train_iter)

(3,
 "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.")

- (2) 데이터 처리 파이프라인 준비 

<hr>

* 어휘집(vocab), 단어 벡터(word vector), 토크나이저(tokenizer)
* 가공되지 않은 텍스트 문자열에 대한 데이터 처리 빌딜 블록
* 일반적인 NLP 데이터 처리
    * 첫번째 단계 : 가공되지 않은 학습 데이터셋으로 어휘집 생성
        => 토큰 목록 또는 반복자 받는 내장 팩토리 함수(factor function) : build_vocab_from_iterator
    * 사용자는 어휘집에 추가할 특수 기호 전달 가능

In [8]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

tokenizer = get_tokenizer('basic_english')

train_iter = AG_NEWS(split='train')

In [9]:
# 토큰 제너레이터 함수 : 데이터 추출하여 토큰화
def yield_tokens(data_iter):
    for _, text in data_iter:
        # 라벨, 텍스트  ---> 텍스트 토큰화
        yield tokenizer(text)


# 단어 사전 생성
vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials=['<unk>'])

## <UNK> 인덱스를 0번으로 지정
vocab.set_default_index(vocab['<unk>'])



In [10]:
vocab(['<unk>','am', 'there', 'example', 'beta'])

[0, 1913, 229, 5297, 2238]

In [11]:
# 텍스트 ===> 정수 인코딩
text_pipeline = lambda x : vocab(tokenizer(x))

# 레이블 ===> 정수 인코딩
label_pipeline = lambda x : int(x) - 1


### (3) 데이터 배치(batch)와 반복자 생성
<hr>

- torch.utils.data.DataLoader : getitem(), len(), 구현한 맵 형태(map-style)
- collate_fn() : DataLoader 로부터 생성된 샘플 배치 함수
    * 입력 : DataLoader에 배치 크기(batch size)가 있는 배치(Batch) 데이터

In [12]:
from torch.utils.data import DataLoader

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
def collate_batch(batch):
    # 배치크기 만큼의 라벨, 텍스트, 오프셋 값 저장 변수
    label_list, text_list, offsets = [], [], [0]

    # 1개씩 뉴스기사, 라벨 추출 해서 저장
    for (_label, _text) in batch:
        # 라벨 인코딩 후 저장
        label_list.append(label_pipeline(_label))

        # 텍스트 인코딩 후 저장
        processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
        text_list.append(processed_text)

        # 텍스트 offset 즉, 텍스트 크기/길이 저장
        offsets.append(processed_text.size(0))


    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)

    return label_list.to(device), text_list.to(device), offsets.to(device)





In [17]:
train_iter = AG_NEWS(split='train')
dataloader = DataLoader(train_iter, batch_size=64, shuffle=False, collate_fn=collate_batch)


In [18]:
num_class = len(set([label for (label, text) in train_iter]))
vocab_size = len(vocab)

print('num class = ',num_class,', vocab_size = ' , vocab_size)

num class =  4 , vocab_size =  95811


In [19]:
for labels, texts, offsets in dataloader:
    print(labels, texts, offsets)
    break

tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2], device='cuda:0') tensor([431, 425,   1,  ...,   1,   1,   1], device='cuda:0') tensor([   0,   29,   71,  111,  151,  194,  242,  289,  338,  426,  457,  501,
         557,  588,  638,  692,  724,  750,  802,  834,  865,  894,  922,  944,
         978, 1008, 1041, 1072, 1100, 1128, 1159, 1185, 1215, 1251, 1285, 1311,
        1343, 1394, 1443, 1512, 1588, 1673, 1725, 1759, 1819, 1882, 1923, 1956,
        1999, 2037, 2055, 2076, 2105, 2137, 2179, 2204, 2223, 2243, 2265, 2325,
        2414, 2461, 2545, 2600], device='cuda:0')


In [ ]:
import torch.nn as nn 
import torch.nn.functional as F


class TextSentiment(nn.Module): 
    def __init__(self, vocab_size, embed_dim, num_class): 
        super(TextSentiment, self).__init__() 
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True) 
        self.fc = nn.Linear(embed_dim, num_class) 
        self.init_weights() 

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = TextSentiment(vocab_size, embed_dim, num_class)
criterion = F.cross_entropy()
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)
